<a href="https://colab.research.google.com/github/BenguBarisBalkan/AIN-311_Project/blob/main/web_scaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install bs4
!pip install lxml

In [79]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [80]:
url = "https://www.spritmonitor.de/en/evaluation/co2_efficient_cars.html"

In [81]:
response = requests.get(url)

In [82]:
if response.status_code == 200:
  html = response.text
  soup = BeautifulSoup(html, "lxml")

else:
  print("Failed")

In [ ]:
print(soup.prettify())

In [ ]:
tbody = soup.find('tbody')
print(tbody)

In [ ]:
for x in tbody:
  print(x)

In [84]:
trs = tbody.find_all('tr')

for tr in trs:
  info = tr.find_all('td')
  print(f"{info[0].text: <5} {info[1].text: <30} {info[2].text: <20} {info[3].text: <20}")

1     Dacia Spring (85)              Electricity          67                  
2     Volkswagen Up! (225)           Electricity          68                  
3     Hyundai IONIQ (205)            Electricity          69                  
4     Volkswagen Golf (200)          Electricity          70                  
5     Renault Twingo (68)            Electricity          71                  
6     Skoda Citigo (58)              Electricity          72                  
7     Smart Fortwo (144)             Electricity          73                  
8     Hyundai Kona (225)             Electricity          76                  
9     Nissan Leaf (169)              Electricity          78                  
10    BMW i3 (253)                   Electricity          78                  
11    Fiat 500 (144)                 Electricity          79                  
12    Renault ZOE (479)              Electricity          80                  
13    Kia Soul (59)                  Electricity    

In [85]:
links = []
for tr in trs:
  info = tr.find_all('td')
  link = "https://www.spritmonitor.de" + info[1].find('a').get('href').replace(' ', '_')
  links.append(link)
  print(f"{link}")

https://www.spritmonitor.de/en/overview/113-Dacia/1958-Spring.html?fueltype=5
https://www.spritmonitor.de/en/overview/50-Volkswagen/1234-Up!.html?fueltype=5
https://www.spritmonitor.de/en/overview/19-Hyundai/1554-IONIQ.html?fueltype=5
https://www.spritmonitor.de/en/overview/50-Volkswagen/452-Golf.html?fueltype=5
https://www.spritmonitor.de/en/overview/41-Renault/374-Twingo.html?fueltype=5
https://www.spritmonitor.de/en/overview/45-Skoda/1289-Citigo.html?fueltype=5
https://www.spritmonitor.de/en/overview/32-Smart/296-Fortwo.html?fueltype=5
https://www.spritmonitor.de/en/overview/19-Hyundai/1638-Kona.html?fueltype=5
https://www.spritmonitor.de/en/overview/33-Nissan/1296-Leaf.html?fueltype=5
https://www.spritmonitor.de/en/overview/6-BMW/1312-i3.html?fueltype=5
https://www.spritmonitor.de/en/overview/16-Fiat/761-500.html?fueltype=5
https://www.spritmonitor.de/en/overview/41-Renault/1302-ZOE.html?fueltype=5
https://www.spritmonitor.de/en/overview/23-Kia/1147-Soul.html?fueltype=5
https://www

In [86]:
with open('./spritmonitor_data/spritmonitor_links.txt', 'w') as file:
  for link in links:
    file.write(link + '\n')

In [87]:
links = []
with open('./spritmonitor_data/spritmonitor_links.txt', 'r') as file:
    for line in file:
        links.append(line.strip())

for link in links:
    print(link)

https://www.spritmonitor.de/en/overview/113-Dacia/1958-Spring.html?fueltype=5
https://www.spritmonitor.de/en/overview/50-Volkswagen/1234-Up!.html?fueltype=5
https://www.spritmonitor.de/en/overview/19-Hyundai/1554-IONIQ.html?fueltype=5
https://www.spritmonitor.de/en/overview/50-Volkswagen/452-Golf.html?fueltype=5
https://www.spritmonitor.de/en/overview/41-Renault/374-Twingo.html?fueltype=5
https://www.spritmonitor.de/en/overview/45-Skoda/1289-Citigo.html?fueltype=5
https://www.spritmonitor.de/en/overview/32-Smart/296-Fortwo.html?fueltype=5
https://www.spritmonitor.de/en/overview/19-Hyundai/1638-Kona.html?fueltype=5
https://www.spritmonitor.de/en/overview/33-Nissan/1296-Leaf.html?fueltype=5
https://www.spritmonitor.de/en/overview/6-BMW/1312-i3.html?fueltype=5
https://www.spritmonitor.de/en/overview/16-Fiat/761-500.html?fueltype=5
https://www.spritmonitor.de/en/overview/41-Renault/1302-ZOE.html?fueltype=5
https://www.spritmonitor.de/en/overview/23-Kia/1147-Soul.html?fueltype=5
https://www

In [89]:
all_cars = []
for url in links:
    car_links = []
    # connect to the first page
    response = requests.get(url)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, "lxml")

    else:
        print("Failed")

    # get the car links from the first page
    trs = soup.find('tbody').find_all('tr')
    for tr in trs:
        info = tr.find_all('td')
        car_link = 'https://www.spritmonitor.de' + info[1].find('a').get('href')
        #print(car_link)
        car_links.append(car_link)

    
    # find next pages' links
    next_page_url = 'https://www.spritmonitor.de' + soup.find('div', class_ = 'paginationControl').find_all('a')[-1].get('href')

    # iterate over next pages until the last
    while True:
        response = requests.get(next_page_url)
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, "lxml")

        else:
            print("Failed")
            break

        # find car links
        trs = soup.find('tbody').find_all('tr')
        for tr in trs:
            info = tr.find_all('td')
            car_link = 'https://www.spritmonitor.de' + info[1].find('a').get('href')
            #print(car_link)
            car_links.append(car_link)
        
        # find next page
        next_link = 'https://www.spritmonitor.de' + soup.find('div', class_ = 'paginationControl').find_all('a')[-1].get('href')
        
        end_p = soup.find('div', class_ = 'paginationControl').find('span', class_ = 'disabled')
        # if the last page has an end_p class, end the loop
        
        if end_p == None:
            next_page_url = next_link
        else:
            break
    
    print(len(car_links))
    all_cars.extend(car_links)

85
225
206
200
68
58
144
225
169
253
144
479
59
106
150
105
81
590
71
150
195
108
225
120
98
63
338
384
136
185
127
77
60
123
85
53
529
54
2191
2443
65
158
74
188
337
255
708
402
78
64


In [90]:

with open('./spritmonitor_data/car_links.txt', 'w') as file:
    for car_link in all_cars:
        file.write(car_link + '\n')